
# Chapter 2

## Python Advanced(2) - Context Manager Annotation

> Keyword - @contextlib.contextmanager, `__enter__`, `__exit__`


 * 가장 대표적인 with 구문 이해
 * Contextlib 데코레이터 사용
 * 코드 직관적, 예외처리 용이성


In [9]:
import contextlib
import time


# Ex1:

@contextlib.contextmanager
def file_writer(filename: str, mode: str):
    f = open(filename, mode)
    yield f  # <-- __enter__
    # __exit__
    f.close()

with file_writer('testfile4.txt', 'w') as f:
    f.write('Context Manager Test4.\nContextlib Test4.')


# Ex2:

@contextlib.contextmanager
def timer(msg: str):
    start = time.monotonic()
    try:
        yield start  # <-- __enter__
    
    # __exit__
    except BaseException as e:
        print(f'Exception {msg}: {e}')
    else:
        print(f'{msg}: {time.monotonic() - start} sec')


with timer('Start Job!') as t:
    print(f'Received start monotonic1: {t}')
    for _ in range(10000000):
        ...

# Received start monotonic1: 902307.203
# Start Job!: 0.2029999999795109 sec

try:
    with timer('Start Job!') as t:
        print(f'Received start monotonic1: {t}')
        for _ in range(10000000):
            ...
        raise Exception('Raise Exception!')
except:
    pass
    
# Received start monotonic1: 902307.406
# Exception Start Job!: Raise Exception!


Received start monotonic1: 911834.5
Start Job!: 0.2029999999795109 sec
Received start monotonic1: 911834.718
Exception Start Job!: Raise Exception!



## Python Advanced(2) - Property(1) - Underscore
> Keyword - access modifier(접근지정자), underscore


 * 다양한 언더스코어 활용
 * 파이썬 접근지정자 설명


In [2]:
# Ex1
# Use underscore
# 1.인터프리터, 2.값 무시, 3.네이밍(국제화, 자릿수 등)


x, _, y = (1, 2, 3)
a, *_, b = (1, 2, 3, 4, 5)

print(x, y)  # 1 3
print(a, b)  # 1 5

for _ in range(10):
    ...

for _, val in enumerate(range(10)):
    ...

print(123_456)            # 123456

print(dir(123_456)[0:5])  # ['__abs__', '__add__', '__and__', '__bool__', '__ceil__']

1 3
1 5
123456
['__abs__', '__add__', '__and__', '__bool__', '__ceil__']


In [3]:

# Ex2
# 접근지정자
#   name : public
#  _name : protected
# __name : private
# 파이썬 -> Public 강제 X, 약속된 규약에 따라 코딩 장려(자유도, 책임감 장려)
#           타 클래스(클래스변수,인스턴스 변수 값 쓰기 장려 안함), -> Naming Mangling
#           타 클래스 __ 접근하지 않는 것이 원칙.

class SampleA:
    
    def __init__(self) -> None:
        self.x = 0
        self.__y = 1
        self._z = 2

a = SampleA()
 
print(f'Ex2 > x: {a.x}')                      # Ex2 > x: 0
# print(f'Ex2 > y: {a.__y}')                  # AttributeError: 'SampleA' object has no attribute '__y'
print(f'Ex2 > z: {a._z}')                     # Ex2 > x: 2
print(f'Ex2 > _SampleA__y: {a._SampleA__y}')  # Ex2 > _SampleA__y: 1



Ex2 > x: 0
Ex2 > z: 2
Ex2 > _SampleA__y: 1


In [4]:

# Ex3
# 메소드 활용 Getter, Setter 작성

class SampleB:
    
    def __init__(self):
        self.x = 0
        self.__y = 0
 
    def get_y(self):
        return self.__y
 
    def set_y(self, value):
        self.__y = value

b = SampleB()

b.x = 1
b.set_y(2)


print('Ex3 > x : {}'.format(b.x))          # Ex3 > x : 1
print('Ex3 > y : {}'.format(b.get_y()))    # Ex3 > y : 2

# 변수 접근 후 수정 부분에서 일관성, 가독성 하락
b._SampleB__y = 343
print('Ex3 > y : {}'.format(b.get_y()))    # Ex3 > y : 343

print('Ex3 >', dir(b)) # _SampleB__y


Ex3 > x : 1
Ex3 > y : 2
Ex3 > y : 343
Ex3 > ['_SampleB__y', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_y', 'set_y', 'x']



## Python Advanced(2) - Property(2) - Getter, Setter
> Keyword - @Property


### 프로퍼티(Property) 사용 장점
1. 파이써닉한 코드
2. 변수 제약 설정
3. Getter, Setter 효과 동등(코드 일관성)
   - 캡슐화-유효성 검사 기능 추가 용이
   - 대체 표현(속성 노출, 내부의 표현 숨기기 가능)
   - 속성의 수명 및 메모리 관리 용이
   - 디버깅 용이
   - Getter, Setter 작동에 대해 설계된 여러 라이브러리(오픈소스) 상호 운용성 증가
  


In [5]:

# Ex1
# Property 활용 Getter, Setter 작성

class SampleA:

    def __init__(self):
        self.x = 0
        self.__y = 0 # private

    @property
    def y(self):
        # print('Called get method.')
        return self.__y
    
    @y.setter
    def y(self, value):
        # print('Called set method.')
        self.__y = value

    @y.deleter
    def y(self):
        print('Called del method.')
        del self.__y

a = SampleA()

a.x = 1
a.y = 2

print('Ex1 > x : {}'.format(a.x))
print('Ex1 > y : {}'.format(a.y))

# a._SampleA__y = 100 # 변수 수정 가능

# 변수 접근 후 수정 부분에서 일관성, 가독성 하락
print('Ex1 >', dir(a)) # _SampleB__y

# deleter 확인
# del a.y
# print('Ex1 >', dir(a))


# Ex2
# Property 활용 제약 조건 추가

class SampleB:

    def __init__(self):
        self.x = 0
        self.__y = 0 # private

    @property
    def y(self):
        # print('Called get method.')
        return self.__y
    
    @y.setter
    def y(self, value):
        # print('Called set method.')
        if value < 0:
            raise ValueError('0 보다 큰 값을 입력하세요.')
        self.__y = value

    @y.deleter
    def y(self):
        print('Called del method.')
        del self.__y

b = SampleB()

b.x = 1
b.y = 10
# b.y = -5 # 예외발생

print('Ex2 > x : {}'.format(b.x))
print('Ex2 > y : {}'.format(b.y))


Ex1 > x : 1
Ex1 > y : 2
Ex1 > ['_SampleA__y', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'x', 'y']
Ex2 > x : 1
Ex2 > y : 10


## Python Advanced(2) - Method Overriding
> Keyword - Overriding, OOP, 다형성


### 메소드 오버라이딩 효과
1. 서브클래스에서 슈퍼(부모)클래스를 호출 후 사용
2. 메소드 재 정의 후 사용 가능
3. 부모클래스의 메소드 추상화 후 사용 가능(구조적 접근)
4. 확장 가능, 다형성(다양한 방식으로 동작)
5. 가독성 증가, 오류가능성 감소, 메소드 이름 절약, 유지보수성 증가 등


In [14]:

# Ex1: Inheritance

class ParentEx1:

    def __init__(self) -> None:
        self.value = 5

    def get_value(self):
        return self.value


class ChildEx1(ParentEx1):
    pass


p1 = ParentEx1()
c1 = ChildEx1()

print('Ex1 > ', p1.get_value())               # 5
print('Ex1 > ', c1.get_value())               # 5
print('Ex1 > ', set(dir(p1)) - set(dir(c1)))  # set()
print('Ex1 > ', set(ParentEx1.__dict__) - set(ChildEx1.__dict__))  # {'__dict__', '__weakref__', '__init__', 'get_value'}



Ex1 >  5
Ex1 >  5
Ex1 >  set()
Ex1 >  {'__dict__', '__weakref__', '__init__', 'get_value'}


In [15]:
# Ex2: Override

class ParentEx2:

    def __init__(self) -> None:
        self.value = 5

    def get_value(self):
        return self.value


class ChildEx2(ParentEx2):
    
    def get_value(self):
        return self.value * 10


p1 = ParentEx2()
c1 = ChildEx2()

print('Ex1 > ', p1.get_value())               # 5
print('Ex1 > ', c1.get_value())               # 50
print('Ex1 > ', set(dir(p1)) - set(dir(c1)))  # set()
print('Ex1 > ', set(ParentEx2.__dict__) - set(ChildEx2.__dict__))  # {'__dict__', '__weakref__', '__init__'}



Ex1 >  5
Ex1 >  50
Ex1 >  set()
Ex1 >  {'__dict__', '__weakref__', '__init__'}


In [17]:
# Ex3: Polymorphism

import datetime

class Logger:

    def log(self, msg):
        print(msg)


class TimestampLogger(Logger):

    def log(self, msg):
        message = "{ts} {msg}".format(ts=datetime.datetime.now(), msg=msg) 
        # super().log(message)
        super(TimestampLogger, self).log(message)


class DateLogger(Logger):

    def log(self, msg):
        message = "{ts} {msg}".format(ts=datetime.datetime.now().strftime('%Y-%m-%d'), msg=msg) 
        super().log(message)
        # super(DateLogger, self).log(message)


parentLogger    = Logger()
timestampLogger = TimestampLogger()
dateLogger      = DateLogger()

parentLogger.log('Called Logger')                # Called Logger
timestampLogger.log('Called Timestamp Logger')   # 2022-01-09 21:23:07.602995 Called Timestamp Logger
dateLogger.log('Called Date Logger')             # 2022-01-09 Called Date Logger



Called Logger
2022-01-09 21:23:07.602995 Called Timestamp Logger
2022-01-09 Called Date Logger


## Python Advanced(2) - Method Overloading
> Keyword - overloading, oop, multiple dispatch


### 메소드 오버로딩 효과
1. 동일 메소드 재정의
2. 네이밍으로 기능 예측
3. 코드 절약, 가독성 향상
4. 메소드 파라미터 기반 호출 방식


In [27]:

# Ex1
# 동일 이름 메소드 사용 예제
# 동적 타입 검사 -> 런타임에 실행(타입 에러가 실행시에 발견)


class SampleA:

    def add(self, x, y):
        return x + y
    
    def add(self, x, y, z):
        return x + y + z

a = SampleA()
# print('Ex 1 > ', a.add(2, 3))  # TypeError: add() missing 1 required positional argument: 'z'


# Ex2
# 동일 이름 메소드 사용 예제
# 자료형에 따른 분기 처리

class SampleB:

    def add(self, datatype, *args):
        if datatype == 'int':
            return sum(args)
        if datatype == 'str':
            return ''.join(args)

b = SampleB()

print('Ex2 > ', b.add('int', 5, 6))      # Ex2 >  11
print('Ex2 > ', b.add('str', '5', '6'))  # Ex2 >  56



# Ex3
# multipledispatch 패키지를 통한 메소드 오버로딩
from multipledispatch import dispatch


class SampleC:

    @dispatch(int, int)
    def add(self, x, y):
        return x + y
    
    @dispatch(int, int, int)
    def add(self, x, y, z):
        return x + y + z

    @dispatch(float, float, float)
    def add(self, x, y, z):
        return x * y * z


c = SampleC()

print('Ex3 > ', c.add(1, 2))           # Ex3 >  3
print('Ex3 > ', c.add(1, 2, 3))        # Ex3 >  6
print('Ex3 > ', c.add(1.5, 2.5, 3.5))  # Ex3 >  13.125

print(c.__getattribute__('add'))       # <dispatched add>
print(type(c.__getattribute__('add'))) # <class 'multipledispatch.dispatcher.MethodDispatcher'>



Ex2 >  11
Ex2 >  56
Ex3 >  3
Ex3 >  6
Ex3 >  13.125
<dispatched add>
<class 'multipledispatch.dispatcher.MethodDispatcher'>
